In [3]:
import tensorflow as tf
import keras
import numpy as np

# Load your trained model
model = keras.models.load_model('models/model1.keras')

# You need to provide this manually!
# Replace X with the correct value (you printed "Maximum length" during training)
max_len = 22  # e.g. max_len = 22

# Use your existing num_to_char (already defined in your notebook!)
# from your previous cell: num_to_char = StringLookup(...).invert()

# Reuse your distortion_free_resize() exactly as defined:
def distortion_free_resize(image, img_size):
    w, h = img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height // 2

    if pad_width % 2 != 0:
        width = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
            [pad_height_top, pad_height_bottom],
            [pad_width_left, pad_width_right],
            [0, 0],
        ],
    )

    image = tf.transpose(image, (1, 0, 2))
    image = tf.image.flip_left_right(image)

    return image

# Reuse your preprocess_image() exactly as defined:
def preprocess_image(image_path, img_size=(128, 32)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)  # Grayscale
    image = distortion_free_resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Final utility function:
def predict_text_from_image(image_path, model, max_len, num_to_char):
    # Preprocess
    img = preprocess_image(image_path)
    img = tf.expand_dims(img, axis=0)  # Add batch dimension

    # Predict
    preds = model.predict(img)

    # Decode (reuse your decode_batch_predictions logic)
    input_len = np.ones(preds.shape[0]) * preds.shape[1]
    results = keras.ops.nn.ctc_decode(preds, sequence_lengths=input_len)[0][0][:, :max_len]

    # Convert to string
    output_text = []
    for res in results:
        res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
        res = (
            tf.strings.reduce_join(num_to_char(res))
            .numpy()
            .decode("utf-8")
            .replace("[UNK]", "")
        )
        output_text.append(res)

    return output_text[0]  # Single image → single result

# Example usage:
# text = predict_text_from_image('path/to/your_image.png', model, max_len, num_to_char)
# print("Predicted text:", text)


TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'handwriting_recognizer', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 128, 32, 1], 'dtype': 'float32', 'sparse': False, 'name': 'image'}, 'registered_name': None, 'name': 'image', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'Conv1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'filters': 32, 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'grou

In [ ]:
text = predict_text_from_image('samples/test1.png', model, max_len, num_to_char)
print("Predicted text:", text)
